Questo notebook integra i dati di ANAC con quelli di IndicePA e contiene una serie di analisi descrittive. 

Output: *data/anac/anac_indicepa.tsv* contiene tutti i bandi di gara avvenuti nel 2017 da parte di PA censite in indicePA.

**Note**
1. Sono stati estratti un totale di 3913440 gare, di cui 1842313 avvenute nell'anno 2017. Si procede all'analisi descrittiva di quest'ultimi bandi.
2. Molti data record contengono valori inconsistenti dovuti a typo (e.g. date con valori errate - 202/02/2016) o ad omissioni da parte delle PA (e.g. codici fiscali di strutture proponenti aventi valori nulli o uguali a *0*, codici fiscali di aggiudicatari offuscati o inesistenti). 


In [1]:
import pandas as pd
import numpy as np
import csv

In [2]:
dir_file = "data/final_data/row_data/"
f_anac = dir_file + "anacDataset_1525953489668.csv"
#df = pd.read_csv(f_anac, sep="\t", error_bad_lines=False, parse_dates=True)
df = pd.read_csv(f_anac, sep="\t", parse_dates=True, dtype= {"cfStrutturaProponente": str})

In [3]:
#ci sono typo in alcune date (e.g. 202/02/2016)
df['dataInizio'] = pd.to_datetime(df['dataInizio'], errors='coerce', format="%d/%m/%Y")
df['dataUltimazione'] = pd.to_datetime(df['dataUltimazione'], errors='coerce', format="%d/%m/%Y")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4067642 entries, 0 to 4067641
Data columns (total 17 columns):
cig                                  object
cfStrutturaProponente                object
denominazioneStrutturaProponente     object
oggetto                              object
sceltaContraente                     object
importoAggiudicazione                float64
importoSommeLiquidate                float64
dataInizio                           datetime64[ns]
dataUltimazione                      datetime64[ns]
jsonPartecipanti                     object
jsonAggiudicatari                    object
totalePartecipanti                   int64
totaleAggiudicatari                  int64
totaleRaggruppamentopartecipanti     int64
totaleRaggruppamentoAggiudicatari    int64
cfPrimoaggiudicatario                object
denominazionePrimoaggiudicatario     object
dtypes: datetime64[ns](2), float64(2), int64(4), object(9)
memory usage: 527.6+ MB


In [5]:
#pulisco i dati dalla string CDATA ed elimino spazi da i cf degli aggiudicatari
df.oggetto = df.oggetto.apply(lambda x :str(x).lower().replace('cdata', ''))
df.cfPrimoaggiudicatario = df.cfPrimoaggiudicatario.str.replace(' ', '')

## Analisi gare 2017

In [7]:
is_2017 = (df['dataInizio'] >= '01/01/2017') & (df['dataInizio'] <= '31/12/2017')
df_2017 = df.loc[is_2017]
df_2017.shape

(1990225, 17)

In [8]:
df_cfPrimoaggiudicatario_count = df_2017[["cfStrutturaProponente", "cfPrimoaggiudicatario"]].groupby(["cfPrimoaggiudicatario"])["cfPrimoaggiudicatario"].agg(["count"]).reset_index(0)
#df_cfPrimoaggiudicatario_count.head(3)
df_cfPrimoaggiudicatario_sum = df_2017[["importoSommeLiquidate","importoAggiudicazione", "cfPrimoaggiudicatario"]].groupby(["cfPrimoaggiudicatario"]).sum().reset_index()
#df_cfPrimoaggiudicatario_sum.head(3)

In [9]:
print("df_cfPrimoaggiudicatario_count", df_cfPrimoaggiudicatario_count.shape)
print("df_cfPrimoaggiudicatario_sum", df_cfPrimoaggiudicatario_sum.shape)

df_cfPrimoaggiudicatario_count (334851, 2)
df_cfPrimoaggiudicatario_sum (334851, 3)


In [10]:
df_denominazione_aggiudicatario = df_2017[["cfPrimoaggiudicatario","denominazionePrimoaggiudicatario"]]
df_denominazione_aggiudicatario = df_denominazione_aggiudicatario.set_index("cfPrimoaggiudicatario")
df_denominazione_aggiudicatario = df_denominazione_aggiudicatario[~df_denominazione_aggiudicatario.index.duplicated(keep='first')]
#df_denominazione_aggiudicatario.head(3)

In [11]:
df_join = df_cfPrimoaggiudicatario_count.set_index('cfPrimoaggiudicatario').join(df_cfPrimoaggiudicatario_sum.set_index('cfPrimoaggiudicatario'))
#df_join.head(3)
df_join.shape

(334851, 3)

In [12]:
#Elimino i duplicati dalle denominazioni
df_denominazione_aggiudicatario = df_denominazione_aggiudicatario[~df_denominazione_aggiudicatario.index.duplicated(keep='first')]
df_denominazione_aggiudicatario.shape

(334852, 1)

In [13]:
final_df = pd.merge(df_denominazione_aggiudicatario, df_join, left_index=True, right_index=True, how='inner')
final_df.shape

(334851, 4)

In [14]:
final_df.head(2)

,denominazionePrimoaggiudicatario,count,importoSommeLiquidate,importoAggiudicazione
cfPrimoaggiudicatario,,,,
03404540712,ASSIGEM SRL,123,567787.98,914616.45
00477710552,Tipolitografia Federici snc ...,3,1150.00,1540.00


## Top aziende, nel 2017, aggiudicatare in termini di gare vinte e importo aggiudicazione 

**Note**

1. Gli aggiudicatari dei bandi di gara possono essere sia singole aziende che raggruppamenti. In quest'ultimo caso il campo *cfPrimoaggiudicatario* e il campo *denominazionePrimoaggiudicatario* avranno valore *Raggruppamento*;
2. Ci sono 74668 bandi di gara vinti da raggruppamenti. Si dedica a questi data record una successiva analisi.

In [15]:
top_aggiuficatari = final_df.sort_values(['count', "importoAggiudicazione"], ascending=False)
top_aggiuficatari.head()

,denominazionePrimoaggiudicatario,count,importoSommeLiquidate,importoAggiudicazione
cfPrimoaggiudicatario,,,,
Raggruppamento,Raggruppamento,78733,1.417035e+09,6.173817e+09
00150470342,Gruppo Spaggiari Parma SpA,15059,7.611607e+06,8.394848e+06
00488410010,TELECOM ITALIA SpA,12058,4.095627e+07,1.320545e+08
02027040019,CDATABORGIONE CENTRO DIDATTICO SRL,7530,3.481685e+06,4.013850e+06
03222970406,MYO SRL,7297,4.179742e+06,6.787292e+06


## Integrazione dati ANAC con IndicePA

Il file *"data/indicePA/indicePA.tsv"* è stato estratto a partire dal notebook *Analisi IndicePA*.

**Note**

- Anno 2017: Su 1.842.313 bandi di gara, solo 1.321.761 hanno codici fiscali di strutture proponenti validi. Questo è dovuto al fatto che alcuni codici fiscali sono errati, mentre altri sono codici fiscali di SPA (non censite in IndicePA). Il dataframe è salvato in *data/anac/anac_indicepa_2017.tsv*
- Tutti gli anni: Su 3.913.440  bandi di gara, 2.682.931 hanno codici fiscali di strutture proponenti validi.

In [16]:
f_indicePA = "data/indicePA/indicePA.tsv"
#df_indicePA = pd.read_csv(f_indicePA, sep="\t")
df_indicePA = pd.read_csv(f_indicePA, sep="\t",dtype={'cf': str})

In [17]:
df_indicePA = df_indicePA.rename(columns={'cf': 'cfStrutturaProponente'})
df_indicePA.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13941 entries, 0 to 13940
Data columns (total 8 columns):
cfStrutturaProponente    13941 non-null object
cod_amm                  13941 non-null object
regione                  13941 non-null object
provincia                13493 non-null object
comune                   13941 non-null object
indirizzo                13941 non-null object
tipologia_istat          13941 non-null object
tipologia_amm            13941 non-null object
dtypes: object(8)
memory usage: 871.4+ KB


In [18]:
df_indicePA.head(2)

,cfStrutturaProponente,cod_amm,regione,provincia,comune,indirizzo,tipologia_istat,tipologia_amm
0,81002250892,istsc_srpc070006,Sicilia,SR,Augusta,"Via Strazzulla, 10",Istituti di Istruzione Statale di Ogni Ordine ...,Pubbliche Amministrazioni
1,83002610844,istsc_agis021005,Sicilia,AG,Ribera,Piazza Zamenhof 1,Istituti di Istruzione Statale di Ogni Ordine ...,Pubbliche Amministrazioni


### ANNO 2017

In [19]:
print(df_indicePA.shape)
print(df_2017.shape)

(13941, 8)
(1990225, 17)


In [20]:
#df_2017_indicePA = pd.merge(df_2017, df_indicePA, left_index=True, right_index=True, how='inner')
df_2017_indicePA = pd.merge(df_2017, df_indicePA, left_on="cfStrutturaProponente", right_on="cfStrutturaProponente", how='inner')
df_2017_indicePA.shape

(1774596, 24)

In [21]:
diff = df_2017.shape[0] - df_2017_indicePA.shape[0]
print("Gare di PA non recensite su indicePA: ", diff)

Gare di PA non recensite su indicePA:  215629


In [22]:
df_2017_indice_left = pd.merge(df_2017, df_indicePA, left_on="cfStrutturaProponente", right_on="cfStrutturaProponente", how='left')
df_2017_indice_left.head()

,cig,cfStrutturaProponente,denominazioneStrutturaProponente,oggetto,sceltaContraente,importoAggiudicazione,importoSommeLiquidate,dataInizio,dataUltimazione,jsonPartecipanti,...,totaleRaggruppamentoAggiudicatari,cfPrimoaggiudicatario,denominazionePrimoaggiudicatario,cod_amm,regione,provincia,comune,indirizzo,tipologia_istat,tipologia_amm
0,Z211AC7EEA,06245760720,SERVIZI POLIFUNZIONALI ECOSOSTENIBILI GIOIA SP...,assicurazione all risk impianto fotovoltaico,23AFFIDAMENTO IN ECONOMIA AFFIDAMENTO DIRETTO,576.0,576.0,2017-01-01,2017-12-31,"{""raggruppamento"":[],""partecipante"":[{""codiceF...",...,0,03404540712,ASSIGEM SRL,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ZC91BDE367,91009990556,COLLEGIO INFRI PROFLI ASS SAN VIG DINFANZIA T...,stampa cartelline con logo del colleg...,23AFFIDAMENTO IN ECONOMIA AFFIDAMENTO DIRETTO,390.0,0.0,2017-02-10,2017-02-20,"{""raggruppamento"":[],""partecipante"":[{""codiceF...",...,0,00477710552,Tipolitografia Federici snc ...,ciavp_0,Umbria,TR,Terni,"Via Galvani, 11","Federazioni Nazionali, Ordini, Collegi e Consi...",Pubbliche Amministrazioni
2,Z971C7B78,02250590995,AGENZIA REGIONALE PER I SERVIZI EDUCATIVI E PE...,affidamento alla ditta dedagroup spa del servi...,17AFFIDAMENTO DIRETTO EX ART 5 DELLA LEGGE N38191,1995.0,0.0,2017-01-01,2017-12-31,"{""raggruppamento"":[],""partecipante"":[{""codiceF...",...,0,Raggruppamento,Raggruppamento,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Z941C74DD1,02250590995,AGENZIA REGIONALE PER I SERVIZI EDUCATIVI E PE...,affidamento alla ditta liberty line srl dei se...,17AFFIDAMENTO DIRETTO EX ART 5 DELLA LEGGE N38191,1300.0,0.0,2017-01-01,2017-12-31,"{""raggruppamento"":[],""partecipante"":[{""codiceF...",...,0,Raggruppamento,Raggruppamento,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ZDE1C74D2C,02250590995,AGENZIA REGIONALE PER I SERVIZI EDUCATIVI E PE...,affidamento alla ditta pesce giancarlo snc del...,17AFFIDAMENTO DIRETTO EX ART 5 DELLA LEGGE N38191,350.0,0.0,2017-01-01,2017-12-31,"{""raggruppamento"":[],""partecipante"":[{""codiceF...",...,0,Raggruppamento,Raggruppamento,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
df_2017_indicePA.head(2)

,cig,cfStrutturaProponente,denominazioneStrutturaProponente,oggetto,sceltaContraente,importoAggiudicazione,importoSommeLiquidate,dataInizio,dataUltimazione,jsonPartecipanti,...,totaleRaggruppamentoAggiudicatari,cfPrimoaggiudicatario,denominazionePrimoaggiudicatario,cod_amm,regione,provincia,comune,indirizzo,tipologia_istat,tipologia_amm
0,ZC91BDE367,91009990556,COLLEGIO INFRI PROFLI ASS SAN VIG DINFANZIA T...,stampa cartelline con logo del colleg...,23AFFIDAMENTO IN ECONOMIA AFFIDAMENTO DIRETTO,390.0,0.00,2017-02-10,2017-02-20,"{""raggruppamento"":[],""partecipante"":[{""codiceF...",...,0,00477710552,Tipolitografia Federici snc ...,ciavp_0,Umbria,TR,Terni,"Via Galvani, 11","Federazioni Nazionali, Ordini, Collegi e Consi...",Pubbliche Amministrazioni
1,Z1818537BE,80007870241,COMUNE DI POZZOLEONE,integrazione impegno di spesa per acquisto ope...,23AFFIDAMENTO IN ECONOMIA AFFIDAMENTO DIRETTO,0.0,1665.24,2017-02-02,2017-02-02,"{""raggruppamento"":[],""partecipante"":[{""codiceF...",...,0,01640430243,LIBRERIA PALAZZO ROBERTI,c_g957,Veneto,VI,Pozzoleone,"Via Roma, 5",Comuni e loro Consorzi e Associazioni,Pubbliche Amministrazioni


In [24]:
df_2017_indicePA.to_csv(dir_file + "anac_indicepa_2017.tsv", sep="\t")

In [25]:
df_2017_indice_left.to_csv(dir_file + "anac_indicepa_2017_leftjoin.tsv", sep="\t")

### Tutti gli anni

In [26]:
df_all_indicePA = pd.merge(df, df_indicePA, left_on="cfStrutturaProponente", right_on="cfStrutturaProponente", how='inner')
diff = df.shape[0] - df_all_indicePA.shape[0]
print("Gare di PA non recensite su indicePA: %d (su %d gare totali)" % (diff, df.shape[0]))

Gare di PA non recensite su indicePA: 549157 (su 4067642 gare totali)


In [27]:
df_all_indicePA.to_csv(dir_file + "anac_indicepa_all.tsv", sep="\t")